In [61]:
import json

import pandas as pd
import petl as etl
from utils.paths import DATA, SITE

In [62]:
TICKET_DATA = SITE / 'themes/ticketing/_data/operational'
TICKET_DATA.mkdir(parents=True, exist_ok=True)

In [63]:
tickets = etl.fromcsv(DATA / 'ticketing/tickets.csv').convertnumbers()

In [64]:
def make_date_index(df):
    df.date=df.date.pipe(pd.DatetimeIndex)
    return df

In [65]:
def resample_to_week(df):
    df = df.resample('W-FRI', on='date').sum()
    df.index.names = ['week_ending']
    return df

In [66]:
by_week = (
    tickets
    .selecteq('geography_type', 'oslaua')
    .aggregate('date', sum, 'count_of_tickets')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .todataframe()
    .pipe(make_date_index)
    .pipe(resample_to_week)
)

In [67]:
weekly_tickets = pd.concat(
    [
        by_week.rename(columns={'value': 'weekly'}),
        by_week.cumsum().rename(columns={'value': 'total'})
    ],
    axis=1
)

In [68]:
weekly_tickets.to_csv(TICKET_DATA / 'weekly_tickets.csv')

In [69]:
geo = tickets.aggregate(['geography_type', 'geography_code'], sum, 'count_of_tickets')

In [70]:
geo.tocsv(TICKET_DATA / 'geography.csv')

In [71]:
orders = (
    etl
    .fromcsv(DATA / 'ticketing/orders.csv')
    .convertnumbers()
    .convert('date', etl.dateparser('%Y-%m-%d'))
)

In [72]:
weekly_orders = (
    orders
    .todataframe()
    .pipe(make_date_index)
    .pipe(resample_to_week)
)


In [73]:
pd.concat(
    [
        weekly_orders.rename(columns={'count_of_orders': 'weekly'}),
        weekly_orders.cumsum().rename(columns={'count_of_orders': 'total'})
    ],
    axis=1
).to_csv(TICKET_DATA / 'weekly_orders.csv')

In [74]:
summary = {
    'orders': orders.values('count_of_orders').sum(),
    'tickets': tickets.selecteq('geography_type', 'osward').values('count_of_tickets').sum(),
}

In [75]:
with open(TICKET_DATA / 'summary.json', 'w') as f:
    json.dump(summary, f)